## Week 5 — Data Transformation

In previous weeks, we learned how to load data, clean missing values, handle categorical variables,  
perform aggregations, and extract meaningful insights.

Now we move to a **new phase** that is part of modern workflows:

> **Preparing the dataset for modeling using feature transformations.**

This includes:
- Scaling numerical features  
- Normalizing values  
- Encoding categorical features  
- Avoiding multicollinearity  

**Dataset Reference:** 🔗 https://archive.ics.uci.edu/dataset/2/adult

---
## SETUP

**Install the required package to load the dataset from UCI repository**

In [67]:
# uncomment the following line to install the required package
#!pip install ucimlrepo

**Import necessary libraries**

In [68]:
import pandas as pd
import numpy as np

from ucimlrepo import fetch_ucirepo

adult = fetch_ucirepo(id=2)
X = adult.data.features
y = adult.data.targets

pd.set_option('display.max_columns', None)

## 1. Load the Dataset

> When loading the Adult Census Income dataset from the UCI repository, you will notice
> that the data is split into **two separate DataFrames**:
>
> - `X` contains all feature columns (Attribute1 … Attribute20)
> - `y` contains the target variable (`class`)
>
> This separation is common in Machine Learning libraries because it clearly
> distinguishes:
>
> - **independent variables** → used to make predictions  
> - **dependent variable** → the value we want to predict
>
> However, for **Exploratory Data Analysis (EDA)**, it is usually more convenient
> to work with a **single unified table**.
>
> Having both features and the target in the same DataFrame simplifies:
>
> - inspecting the overall structure  
> - checking distributions  
> - computing correlations
> - detecting missing values  
> - visualizing relationships between variables
>
> To prepare for EDA, we will **concatenate** the two parts into one unified table.
>
> ### Concatenating DataFrames
>
> The simplest way to combine `X` and `y` is with `pd.concat`, which allows us to
> join DataFrames **side-by-side** using `axis=1`:
>
> - `pd.concat([...])` → specifies the DataFrames to combine  
> - `axis=1` or `axis='columns'` → concatenate **column-wise**, placing the
>   target column next to the features  
>
> **Example:**
>
> ```python
> df = pd.concat([df_1, df_2], axis=1)
>
> # or equivalently
>
> df = pd.concat([df_1, df_2], axis="columns")
> ```
>
> ### What about `axis=0` or `axis='rows'`?
>
> - This stacks DataFrames **row-wise**, one on top of the other.  
> - It requires both DataFrames to have the **same columns**.  
> - Therefore it is *not* appropriate for joining `X` and `y`.

---

### Q1.1 Verify both datasets which are separated DataFrames `X` and `y`, then concatenate them into one using `pd.concat`.


In [69]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [70]:
y.head()

,income
0,<=50K
1,<=50K
2,<=50K
3,<=50K
4,<=50K


In [71]:
df = pd.concat([X, y], axis=1)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Data Dictionary
>
>Below is the official data dictionary for the **Adult Census Income** dataset (UCI ML Repository).  
>
>Unlike the German Credit dataset, the Adult dataset already includes descriptive column names.
>However, several columns contain coded categories, ambiguous meanings, or missing values masked as `" ?"`,  
>which we will address in the cleaning and preprocessing stages.
>
>| Variable Name     | Role    | Type         | Demographic     | Description                                                                                               | Units | Missing Values |
>|-------------------|---------|--------------|-----------------|-----------------------------------------------------------------------------------------------------------|--------|----------------|
>| age               | Feature | Integer      | Age             | Age of the individual                                                                                     | years  | no             |
>| workclass         | Feature | Categorical  | Income/Employment | Employment status (Private, Self-emp-not-inc, Federal-gov, … )                                           |        | yes (encoded as `" ?"`) |
>| fnlwgt            | Feature | Integer      | —               | Final sampling weight (used by US Census Bureau)                                                          |        | no             |
>| education         | Feature | Categorical  | Education       | Highest level of education achieved (Bachelors, HS-grad, Some-college, …)                                 |        | no             |
>| education-num     | Feature | Integer      | Education       | Numerical representation of education level                                                                |        | no             |
>| marital-status    | Feature | Categorical  | Other           | Marital status (Married, Divorced, Never-married, …)                                                      |        | no             |
>| occupation        | Feature | Categorical  | Employment      | Type of occupation (Tech-support, Sales, Exec-managerial, …)                                              |        | yes (encoded as `" ?"`) |
>| relationship      | Feature | Categorical  | Other           | Relationship of the individual to their household                                                          |        | no             |
>| race              | Feature | Categorical  | Race            | Race group (White, Black, Asian-Pac-Islander, …)                                                           |        | no             |
>| sex               | Feature | Binary       | Sex             | Biological sex (Male, Female)                                                                              |        | no             |
>| capital-gain      | Feature | Integer      | —               | Capital gain from investment income                                                                        | USD    | no             |
>| capital-loss      | Feature | Integer      | —               | Capital loss from investment income                                                                        | USD    | no             |
>| hours-per-week    | Feature | Integer      | Employment      | Working hours per week                                                                                     | hours  | no             |
>| native-country    | Feature | Categorical  | Other           | Country of origin                                                                                          |        | yes (encoded as `" ?"`) |
>| income            | Target  | Binary       | Income          | Income category: `<=50K` or `>50K`                                                                         |        | no             |
---
### Q1.2 Obtain the `.info()` from the Dataset:

>Investigate the datatypes of each column. Are they appropriate?

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


### Q1.3 Obtain descriptive statistics using `.describe()`

In [73]:
df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


### Q1.4 Investigate how many missing values are in each column

In [74]:
df.isnull().sum()

age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
income              0
dtype: int64

## 2. Clean the Dataset

Before we proceed any further, it is necessary to ensure that the dataset is **consistent** and **clean**.

> Even though the Adult Income dataset already has descriptive columns,  
> it still contains issues that must be addressed before transformation:
>
> - missing values
> - categorical values with inconsistent spacing  
> - mixed types inside categorical columns  
> - skewed numerical variables  
> - redundant or correlated columns
___
### Q2.1. Create a categorical list containing the columns of the appropriate dtype.

In [75]:
categorical = df.select_dtypes(include=object).columns.to_list()
categorical

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country',
 'income']

### Q2.2. Create a numerical list containing the columns of the appropriate dtype.

In [76]:
numerical = df.select_dtypes(include=np.number).columns.to_list()
numerical

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week']

### Q2.3. For every categorical column verify the unique values

In [77]:
for col in categorical:
    print(col, '\n', df[col].value_counts(), end='\n\n')

workclass 
 workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
?                    1836
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64

education 
 education
HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: count, dtype: int64

marital-status 
 marital-status
Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse           37
Name: count, dtype: int64

occupation 
 occupation
Prof-specialty  

In [78]:
df.isnull().sum()

age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
income              0
dtype: int64

In [79]:
# .unique will reveal nans too
for col in categorical:
    print(col, '\n', df[col].unique(), end='\n\n')

workclass 
 ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked' nan]

education 
 ['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']

marital-status 
 ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']

occupation 
 ['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' '?'
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv' nan]

relationship 
 ['Not-in-family' 'Husband' 'Wife' 'Own-child' 'Unmarried' 'Other-relative']

race 
 ['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']

sex 
 ['Male' 'Female']

native-country 
 ['United-States' 'Cuba' 'Jamaica' 'India' '?' 'Mexico' 'South'
 'Puerto-Rico'

### Q2.4. Clean the `income` column:
> - Use `df["col"].str.replace(".", "", regex=False)`
> - `regex=False` is to guarantee that you are not going to remove all characters

In [80]:
df["income"] = df["income"].str.replace(".", "", regex=False)
df['income'].value_counts()

income
<=50K    37155
>50K     11687
Name: count, dtype: int64

### Q2.5 Replace `"?"` with proper `NaN` in all categorical columns

> As seen in the unique values above, several categorical columns use `"?"`  
> instead of `NaN` to mark missing entries.
>
> Replace **all occurrences** of `"?"` with `np.nan`.
>
>**Example in specific column:**
>```python
>df['col'].replace("?", np.nan, inplace=True)
>```
>**Example in the whole DataFrame:**
>```python
>df.replace("?", np.nan, inplace=True)
>```
___

In [81]:
df.replace('?', np.nan, inplace=True)

In [82]:
df.isnull().sum()

age                  0
workclass         2799
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     857
income               0
dtype: int64

### Q2.6. Create a `categoricals_with_nans` list containing categorical columns with missing values.

In [83]:
df["income"] = df["income"].str.replace(".", "", regex=False).str.strip()
df['income'].value_counts()


income
<=50K    37155
>50K     11687
Name: count, dtype: int64

In [84]:
categoricals_with_nans = [col for col in categorical if df[col].isnull().sum() > 0]
categoricals_with_nans

['workclass', 'occupation', 'native-country']

### Q2.7. Verify the proportion of each column from the list of categoricals with missing values

In [85]:
for col in categoricals_with_nans:
    print(col, '\n', df[col].value_counts(), end='\n\n')

workclass 
 workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64

occupation 
 occupation
Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: count, dtype: int64

native-country 
 native-country
United-States                 43832
Mexico                          951
Philippines                     295
Germany                         206
Puerto-Rico                     184
Canada                          182
El-Salvador                     155
India                           151
Cuba    

### Q2.8. Verify the percentage of missing values in `categoricals_with_nans`

In [86]:
for col in categoricals_with_nans:
    missing_pct = df[col].isna().mean() * 100
    print(f"{col}: {missing_pct:.2f}% missing")

workclass: 5.73% missing
occupation: 5.75% missing
native-country: 1.75% missing


### Q2.9 Decide the imputation strategy for:
>- `workclass`
>- `occupation`
>- `native-country`
>
>**Example of imputation with mode using `.fillna(df[col].mode()[0])`.**
>```python
>   df['col'] = df['col'].fillna(df['col'].mode()[0])
>```
>
>**Example dropping rows with missing values:**
>```python
>   df = df.dropna(subset=["col1", "col2", "..."])
>```
>
>**Example of group-based imputation:**
>```python
>   def group_impute_mode(df, target_col, group_col):
>       """
>       Impute missing values in `target_col` using the mode of each group defined by `group_col`.
>       """
>       group_modes = {}
>
>       # Compute mode per group
>       for group_value, group_df in df.groupby(group_col):
>           mode_val = group_df[target_col].mode()[0]
>           group_modes[group_value] = mode_val
>
>       # Apply the imputation group-wise
>       for group_value, mode_val in group_modes.items():
>           mask = (df[group_col] == group_value)
>           df.loc[mask, target_col] = df.loc[mask, target_col].fillna(mode_val)
>
>       return df
>```
___
**Run the following cell if you decide to use this function...**

**You still need to decide the `target_col` and the `group_col`**

In [87]:
def group_impute_mode(df, target_col, group_col):
    """
    Impute missing values in `target_col` using the mode
    of each group defined by `group_col`.
    If a group has all NaNs, it falls back to the global mode.
    """
    # Global mode (fallback)
    global_mode = df[target_col].mode()[0]

    # Compute mode per group
    group_modes = (
        df.groupby(group_col)[target_col]
          .agg(lambda x: x.mode()[0] if not x.mode().empty else global_mode)
    )

    # Map group modes to rows
    df[target_col] = df[target_col].fillna(
        df[group_col].map(group_modes)
    )

    return df



In [88]:
# 1. workclass → global mode
df['workclass'] = df["workclass"].fillna(df["workclass"].mode()[0])

# 2. occupation → group-based (by education)
df = group_impute_mode(df, target_col="occupation", group_col="education")

# 3. native-country → drop missing rows
df = df.dropna(subset=["native-country"])

### 🤷‍♂️ As discussed in previous classes, all strategies have disadvantages and may introduce bias

| Column           | Your Strategy          | Evaluation                            |
| ---------------- | ---------------------- | ------------------------------------- |
| `workclass`      | Mode                   | Simple, effective                  |
| `occupation`     | Group mode (education) | Excellent & realistic               |
| `native-country` | Drop missing           | Acceptable when missing % is small |


## 3. Why Data Transformation?
>
> Raw datasets usually **cannot** be directly fed into models.
>
> **Many algorithms expect features to be:**
> - on comparable scales  
> - numerical  
> - free of redundant or perfectly correlated dimensions  
> - consistent between train and test sets  
>
> **In practice, real-world datasets contain:**
> - variables measured in different units  
> - skewed distributions  
> - categorical labels  
> - missing values  
> - outliers  
>
> **Transformations help ensure:**
>
>- numerical stability  
>- faster gradient convergence  
>- better performance in distance-based models  
>- reduced bias in parameter estimation  
>- interpretability and model consistency  
>- compatibility with modern pipelines  
___
>
>## Label Encoding
>
>Label Encoding consists of assigning a **numeric code** to each category.
>
>**Example:**
>
>```python
>   education_map = {
>        "Preschool": 0,
>        "1st-4th": 1,
>        "5th-6th": 2,
>        "7th-8th": 3,
>        "9th": 4,
>        "10th": 5,
>        "11th": 6,
>        "12th": 7,
>        "HS-grad": 8,
>        "Some-college": 9,
>        "Assoc-voc": 10,
>        "Assoc-acdm": 11,
>        "Bachelors": 12,
>        "Masters": 13,
>        "Doctorate": 14,
>        "Prof-school": 15
>    }
>
>    df["education_encoded"] = df["education"].map(education_map)
>```
>When **Label Encoding** is appropriate?
> - When a categorical feature has a natural order:
>
>   - Education level
>
>   - Satisfaction scores
>
>   - Risk levels
>
>   - Stage or level progressions
>
>In these cases, the assigned numbers represent rank, not category labels.
___
### Q3.1. Use the **mapping dictionary** below to `Label Encode` the `education` column

In [89]:
education_map = {
    "Preschool": 0,
    "1st-4th": 1,
    "5th-6th": 2,
    "7th-8th": 3,
    "9th": 4,
    "10th": 5,
    "11th": 6,
    "12th": 7,
    "HS-grad": 8,
    "Some-college": 9,
    "Assoc-voc": 10,
    "Assoc-acdm": 11,
    "Bachelors": 12,
    "Masters": 13,
    "Prof-school": 14,
    "Doctorate": 15
}

In [90]:
df["education_encoded"] = df["education"].map(education_map)

df[["education", "education_encoded"]].value_counts()

education     education_encoded
HS-grad       8                    15586
Some-college  9                    10723
Bachelors     12                    7815
Masters       13                    2579
Assoc-voc     10                    2038
11th          6                     1800
Assoc-acdm    11                    1579
10th          5                     1372
7th-8th       3                      931
Prof-school   14                     808
9th           4                      745
12th          7                      635
Doctorate     15                     562
5th-6th       2                      490
1st-4th       1                      240
Preschool     0                       82
Name: count, dtype: int64

## One-Hot Encoding (OHE)

> After understanding Label Encoding, we now need a method that works for  
> categorical variables **without natural order**.  
>
> Assigning integers to categories like `"Red" → 0`, `"Blue" → 1`, `"Green" → 2`  
> would incorrectly imply:
>
> - Blue > Red  
> - Green > Blue  
>
> which creates a **fake ordering relationship** that does not exist.
>
> To avoid this problem, we use **One-Hot Encoding (OHE)**.
___
### What is One-Hot Encoding?
>
> One-Hot Encoding converts each category into a **binary indicator column**.
>
> Suppose the column **`color`** contains the following categories:
>
> - <span style="color:#D62828; font-weight:bold;">Red</span>  
> - <span style="color:#1D3557; font-weight:bold;">Blue</span>  
> - <span style="color:#2A9D8F; font-weight:bold;">Green</span>  
>
> Using **One-Hot Encoding**, each category becomes its own variable.
>
> **Assume the input column `color`:**
>
> | color |
> |-------|
> | <span style="color:#D62828; font-weight:bold;">Red</span> |
> | <span style="color:#1D3557; font-weight:bold;">Blue</span> |
> | <span style="color:#2A9D8F; font-weight:bold;">Green</span> |
>
>**Example using One-Hot Encoding with `pd.get_dummies()`**
>
> ```python
>   df_encoded = pd.get_dummies(df, columns=list_of_columns, drop_first=True, dtype=type)
> ```
>
>**Expected result:**
> | color_Blue | color_Green |
> |------------|--------------|
> |     0      |      0       |
> |     1      |      0       |
> |     0      |      1       |
>
> **Why is the first category omitted?**
>
> Because dropping one dummy avoids the  <span style="color:#E76F51; font-weight:bold;">dummy variable trap</span> caused by **perfect multicollinearity**, where:
>
> ```
> Red = 1 − (Blue + Green)
> ```
>
> **Now every row still uniquely identifies its color, without redundancy.**
___
### Q3.2. Create a DataFrame named `df_encoded` using `OHE` on the remaining categorical columns:
>- Do not forget that `education` column must not be included

In [91]:
categorical = df.select_dtypes(include="object").columns.tolist()
categorical

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country',
 'income']

In [92]:
categorical.remove('education')

'education' in categorical

False

In [93]:
# alternatively using list comprehension
# categorical = [col for col in categorical if col != "education"]

In [94]:
# this will present the subset of columns in the list
df[categorical].head()

,workclass,marital-status,occupation,relationship,race,sex,native-country,income
0,State-gov,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K


In [95]:
# notice the values, they are booleans by default - lets redo this by making these values as int since the dtype is more friendly to ML algorithms
df_encoded = pd.get_dummies(df[categorical], drop_first=True)
df_encoded

,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income_>50K
0,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [96]:
 # ensures 0/1 integers instead of booleans
df_encoded = pd.get_dummies(df[categorical], drop_first=True, dtype=int)
df_encoded

,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income_>50K
0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
48838,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
48839,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
48840,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


### Q3.3. Verify if the original columns are still in the DataFrame
>- Use `occupation` column for example

In [97]:
'ocupation' in df_encoded

False

### Q3.4. Verify the number of columns after the `OHE`

In [98]:
# it increased by a lot right?
df_encoded.shape

(47985, 77)

### Q3.5. Concatenate `df_encoded` with the `education` encoded column

In [99]:
df_encoded = pd.concat([df_encoded, df['education']], axis=1)
df_encoded

,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income_>50K,education
0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Bachelors
1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Bachelors
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,HS-grad
3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,11th
4,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bachelors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Bachelors
48838,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,HS-grad
48839,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Bachelors
48840,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Bachelors


### Q3.6 Verify if there are **outliers** in `numerical` columns using the `IQR method`

> To detect outliers in a numerical column, we can use the **Interquartile Range (IQR) method**.
> The IQR represents the spread of the middle 50% of the data.
>
> The formula works as follows:
>
> - Compute the 1st quartile (Q1) → 25th percentile  
> - Compute the 3rd quartile (Q3) → 75th percentile  
> - Compute the **IQR**:
>
> $$
> \text{IQR} = Q3 - Q1
> $$
>
> Outliers are any observations outside the following bounds:
>
> $$
> \text{Lower Bound} = Q1 - 1.5 \times \text{IQR}
> $$
> $$
> \text{Upper Bound} = Q3 + 1.5 \times \text{IQR}
> $$
>
> Values smaller than the lower bound or greater than the upper bound are considered **outliers**.
>
> Now, define a function that verifies whether a column contains outliers:
>
>```python
>def verify_outliers(df: pd.DataFrame, col: str) -> bool:
>    q1 = df[col].quantile(0.25)
>    q3 = df[col].quantile(0.75)
>    iqr = q3 - q1
>    # continue from here
>```
>
>Return a bool from the function and apply it on every `numerical` column


In [100]:
def verify_outliers(df: pd.DataFrame, col: str) -> bool:
    # Compute quartiles
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1

    # Define bounds
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Return True if outliers exist, else False
    return True if ((df[col] < lower_bound) | (df[col] > upper_bound)).any() else False


In [101]:
for col in numerical:
    has_outliers = verify_outliers(df, col)
    print(f"{col}: {has_outliers}")

age: True
fnlwgt: True
education-num: True
capital-gain: True
capital-loss: True
hours-per-week: True


<details>
<summary><h3>Can we also use the IQR method to remove outliers?</h3></summary>

> Yes, the same mathematical rule used to *detect* outliers can also be used
> to *remove* them.  
>
> Once we compute the lower and upper bounds:
>
> $$
> \text{Lower Bound} = Q1 - 1.5 \times \text{IQR}
> $$
> $$
> \text{Upper Bound} = Q3 + 1.5 \times \text{IQR}
> $$
>
> We can simply filter the DataFrame to keep only the values **within these limits**.
>
> This is known as **IQR-based outlier removal** and is one of the most common
> preprocessing techniques in data cleaning, especially for algorithms that are 
> sensitive to extreme values.
>
> Example function to *remove* outliers from a column:
>
> ```python
> def remove_outliers_iqr(df: pd.DataFrame, col: str) -> pd.DataFrame:
>     q1 = df[col].quantile(0.25)
>     q3 = df[col].quantile(0.75)
>     iqr = q3 - q1
>
>     lower = q1 - 1.5 * iqr
>     upper = q3 + 1.5 * iqr
>
>     return df[(df[col] >= lower) & (df[col] <= upper)]
> ```
>
___
<div style="background-color:#f2f2f2; padding:12px; border-left:4px solid #d9534f; border-radius:4px; margin:10px 0;">
<strong>⚠️ NOTE:</strong> REMOVING OUTLIERS IS NOT ALWAYS RECOMMENDED.<br>
It depends on the context and whether extreme values are real observations or measurement errors.<br>
In credit scoring datasets like this one, outliers may represent important patterns of risk.
</div>


</details>

### Transforming Numerical Features
>
>Before applying more advanced preprocessing techniques, we must ensure that our **numerical features are properly transformed**.
>
>**Raw numerical variables typically present issues such as:**
>
>- very different scales  
>- extreme outliers  
>- heavy-tailed shapes  
>- inconsistent units  
>
>These problems can negatively impact both **data analysis** and **modeling**.
___
### Why Numerical Transformation Matters
>
>**Many algorithms assume that:**
>
> - features are on comparable scales  
> - values are not dominated by extreme outliers  
> - distributions are not extremely skewed  
> - distances between samples are meaningful  
>
>**However, real-world datasets often include:**
>
>- **large scale differences**  
>   - e.g., `capital-gain` ranges up to 100,000+  
>   - e.g., `hours-per-week` ranges from 1 to ~60  
>
>- **strong skewness**  
>   - e.g., most people have zero capital gain or loss
>
>
>**Numerical transformations help ensure:**
>
>- numerical stability  
>- consistency between features  
>- reduction of bias introduced by outliers  
>- improved interpretability  
>- better suitability for downstream ML tasks  
---
### Standardization (Z-Score Scaling)
>
> Standardization rescales a feature so that it has:
>
> - mean = **0**  
> - standard deviation = **1**
>
>**Mathematically:**
>
>$$
>z = \frac{x - \mu}{\sigma}
>$$
>
>**Key characteristics:**
>
>- preserves the **shape** of the distribution  
>- keeps outliers (does not remove them)  
>- spreads values around zero  
>- widely used in:  
>  - Linear / Logistic Regression  
>  - Neural Networks  
>  - PCA  
>  - SVM
>
>**Example:**
>
>```python
>   def standardize(column: pd.Series) -> pd.Series:
>      """
>      Applies Z-score standardization to a numerical column.
>      (x - mean) / std
>      """
>      mean = column.mean()
>      std = column.std()
>      return (column - mean) / std
>
>   df_standard = pd.DataFrame()
>   for col in numericals:
>       df_standard[col] = standardize(df[col])
>
>   df_standard.head()
>```
>
>**Alternatively, you can use `sklearn.preprocessing.StandardScaler` for more efficient scaling.**
>
>```python
>   from sklearn.preprocessing import StandardScaler
>   scaler = StandardScaler()
>   df_standard = pd.DataFrame(scaler.fit_transform(df[numericals]), columns=numericals)
>```
___
### Q3.7. Create a new DataFrame named `df_standard` containing the standardized versions of the `numerical` columns
>- `age` **-> Good candidate for scaling**
>   - Moderate range (17–90)
>   - No extreme outliers
___
>- `fnlwgt` **→ Strong candidate for scaling**
>   - Very large magnitude (20,000 to over 1,000,000)  
>   - Extremely skewed and dominates the dataset if not scaled  
>   - Standardization reduces magnitude-related bias  
---
>- `education-num` **→ Do *NOT* standardize**
>   - Encodes an **ordinal** variable (1 to 16)  
>   - Small range and meaningful progression  
>   - Scaling removes interpretability without adding value  
---
>- `capital-gain` **→ Do *NOT* scale raw values**
>   - 99% of observations are **zero**  
>   - Very few extremely large values  
---
>- `capital-loss` **→ Do *NOT* scale raw values**
>  - Same skewness behavior as `capital-gain`  
>  - Mostly zeros with rare large values
___

In [102]:
cols_to_standardize = ["age", "fnlwgt", "hours-per-week"]

In [103]:
def standardize(column: pd.Series) -> pd.Series:
    """
    Applies Z-score standardization to a numerical column: (x - mean) / std
    """
    mean = column.mean()
    std = column.std()
    return (column - mean) / std

In [104]:
df_standard = pd.DataFrame()

for col in cols_to_standardize:
    df_standard[col] = standardize(df[col])

In [105]:
df_standard.head()

,age,fnlwgt,hours-per-week
0,0.026127,-1.059734,-0.033208
1,0.827315,-1.004941,-2.212196
2,-0.046708,0.246330,-0.033208
3,1.045820,0.426691,-0.033208
4,-0.775060,1.407095,-0.033208


In [106]:
# optionally, you could mantain the original and the standardized
df_standard_full = df.copy()
df_standard_full[cols_to_standardize] = df_standard
df_standard_full.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,education_encoded
0,0.026127,State-gov,-1.059734,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,-0.033208,United-States,<=50K,12
1,0.827315,Self-emp-not-inc,-1.004941,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,-2.212196,United-States,<=50K,12
2,-0.046708,Private,0.246330,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,-0.033208,United-States,<=50K,8
3,1.045820,Private,0.426691,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,-0.033208,United-States,<=50K,6
4,-0.775060,Private,1.407095,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,-0.033208,Cuba,<=50K,12


But, should we apply it to every column?

| Column          | Standardize? | Why?                                    |
|-----------------|--------------|------------------------------------------|
| age             | ✔️ Yes       | Normal range; good candidate             |
| fnlwgt          | ✔️ Yes       | Huge scale; dominates gradients          |
| hours-per-week  | ✔️ Yes       | Moderate range; helpful for ML           |
| education-num   | ❌ No        | Ordinal meaning; should not lose scale   |
| capital-gain    | ❌ No        | Highly skewed → log-transform instead    |
| capital-loss    | ❌ No        | Highly skewed → same as above            |


### Q3.8 Concatenate `df_standard` with the rest of the dataset (`df_encoded`) to create the final cleaned DataFrame named `df_final`

In [107]:
df_final = pd.concat([df_standard, df_encoded], axis=1)
df_final.head()

,age,fnlwgt,hours-per-week,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income_>50K,education
0,0.026127,-1.059734,-0.033208,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Bachelors
1,0.827315,-1.004941,-2.212196,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Bachelors
2,-0.046708,0.246330,-0.033208,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,HS-grad
3,1.045820,0.426691,-0.033208,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,11th
4,-0.775060,1.407095,-0.033208,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bachelors


### Q3.9 Export the dataset to a `csv` file as `cleaned_adult_income.csv`

In [108]:
df_final.to_csv('cleaned_adult_income.csv')

>### Next class we will introduce `Preprocessing Pipelines` using `scikit-learn`
>**This will allow us to combine all the steps above into a single reusable workflow.**